<a href="https://colab.research.google.com/github/fsilvino/INE5644-data-mining-projeto-final/blob/master/eda/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Análise exploratória de dados e preparação de dados por Flávio Silvino
import pandas as pd
import numpy as np
import math

In [ ]:
# Inicialmente o carregamento havia sido feito mais simples:
# Depois fui incrementando com as descobertas feitas durante a exploração
df = pd.read_csv('../datasets/openDataSUS/INFLUD-21-09-2020.csv', delimiter=';')

In [ ]:
# número de linhas e colunas
df.shape

In [ ]:
for i, name in enumerate(df.columns.values):
    if i in (15,60,62,63,64,92,94,108,117,121,123):
        print(str(i) + ' - ' + name + ': ')
        print('dtype: ', df[name].dtype)
        print('Valores: ', end='')
        print(df[name].unique())
        print()

In [ ]:
pd.Series(df['PAC_COCBO'].unique().astype(str)).sort_values(key=lambda x: x.str.lower(), ascending=False).head(5).to_markdown(tablefmt='html')

In [ ]:
# Verificando quais são as colunas de data para converter para datetime
df.filter(regex='^DT_', axis=1).head(5)

In [ ]:
# Indentifiquei que:
#    - As colunas que começam com DT_ são do tipo data e estão no formato dd/mm/aaaa
#    - As colunas 'FLUASU_OUT', 'FLUBLI_OUT', 'CLASSI_OUT', 'LO_PS_VGM', 'OUT_ANIM' são do tipo string
#    - A coluna 'PAC_COCBO' possui um valor inválido 'XXX', então estou convertendo para int e os valores inválidos estão setados para NA
#    - A coluna 'COD_IDADE' possui um valor inválido '20-1', então também estou convertendo para int e os valores inválidos setados para NA
#    - Várias colunas utilizam o código 9 para Ignorado, ou seja, não se sabe o valor pois não foi informado (serão tratados como NA)
to_date_parser = lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='coerce')
to_numeric_parser = lambda x: pd.to_numeric(x, errors='coerce')

# lendo o dataset
df = pd.read_csv('../datasets/openDataSUS/INFLUD-21-09-2020.csv', delimiter=';',
                 date_parser=to_date_parser,
                 parse_dates=['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_UT_DOSE', 'DT_VAC_MAE',
                              'DT_DOSEUNI', 'DT_1_DOSE', 'DT_2_DOSE', 'DT_ANTIVIR', 'DT_INTERNA',
                              'DT_ENTUTI', 'DT_SAIDUTI', 'DT_RAIOX', 'DT_COLETA', 'DT_PCR',
                              'DT_EVOLUCA', 'DT_ENCERRA', 'DT_DIGITA', 'DT_VGM', 'DT_RT_VGM',
                              'DT_TOMO', 'DT_RES_AN', 'DT_CO_SOR', 'DT_RES'],
                 na_values={'CS_SEXO': '9', 'CS_GESTANT': '9', 'CS_RACA': '9', 'CS_ESCOL_N': '9', 'CS_ZONA': '9', 'HISTO_VGM': '9', 'SURTO_SG': '9', 'NOSOCOMIAL': '9', 'AVE_SUINO': '9',
                            'FEBRE': '9', 'TOSSE': '9', 'GARGANTA': '9', 'DISPNEIA': '9', 'DESC_RESP': '9', 'SATURACAO': '9', 'DIARREIA': '9', 'VOMITO': '9', 'DOR_ABD': '9', 'FADIGA': '9',
                            'PERD_OLFT': '9', 'PERD_PALA': '9', 'OUTRO_SIN': '9', 'FATOR_RISC': '9', 'PUERPERA': '9', 'CARDIOPATI': '9', 'HEMATOLOGI': '9', 'SIND_DOWN': '9', 'HEPATICA': '9',
                            'ASMA': '9', 'DIABETES': '9', 'NEUROLOGIC': '9', 'PNEUMOPATI': '9', 'IMUNODEPRE': '9', 'RENAL': '9', 'OBESIDADE': '9', 'OUT_MORBI': '9', 'VACINA': '9',
                            'MAE_VAC': '9', 'M_AMAMENTA': '9', 'ANTIVIRAL': '9', 'HOSPITAL': '9', 'UTI': '9', 'SUPORT_VEN': '9', 'RAIOX_RES': '9', 'TOMO_RES': '9', 'AMOSTRA': '9',
                            'TP_AMOSTRA': '9', 'RES_AN': '9', 'POS_AN_FLU': '9', 'POS_AN_OUT': '9', 'PCR_RESUL': '9', 'POS_PCRFLU': '9', 'POS_PCROUT': '9', 'TP_AM_SOR': '9', 'EVOLUCAO': '9'},
                 converters={'PAC_COCBO': to_numeric_parser, 'COD_IDADE': to_numeric_parser},
                 dtype={'FLUASU_OUT': str, 'FLUBLI_OUT': str, 'CLASSI_OUT': str, 'LO_PS_VGM': str, 'OUT_ANIM': str})

In [ ]:
# Calculando percentual de valores faltantes para cada coluna
def calcularFaltantes(dataFrame):
    registros = dataFrame.shape[0]
    return dataFrame.apply(lambda x: sum(x.isnull()) / registros, axis=0)

def listarFaltantes(faltantes, percentual):
    display(faltantes.loc[faltantes > percentual].sort_values(ascending=False).to_markdown(tablefmt="html"))
    
def calcularEListarFaltantes(dataFrame, percentual):
    faltantes = calcularFaltantes(dataFrame)
    listarFaltantes(faltantes, percentual)

In [ ]:
# Mostrando o percentual de valores faltantes das colunas com mais de 90% de seus registros em branco
calcularEListarFaltantes(df, 0.9)

In [ ]:
# verificando quantos resultados de teste de COVID-19 estão preenchidos (depois verificou-se que existem mais colunas com este dado)
df['RES_IGM'].count()

In [ ]:
df.head()

In [ ]:
# Verificando o período dos dados
print("min:", df['DT_NOTIFIC'].min())
print("max:", df['DT_NOTIFIC'].max())

In [ ]:
# Existe um valor estranho (que parece incorreto: 20-1) que impede a conversão para inteiro na coluna COD_IDADE. Poderíamos remover ou tratar este valor,
# porém como não sabemos para que serve este campo e não encontramos ele no dicionário de dados, estamos descartando-o.
# Além disso, outras colunas que, no nosso entendimento, não serão importantes para o objetivo e escopo deste projeto estão sendo descartadas.
# Assim como colunas que possuem mais de 90% de seus registros com valor em branco (faltante) também foram removidas.
df = df.drop(['COD_IDADE', 'SEM_NOT', 'OBES_IMC', 'DT_UT_DOSE', 'MAE_VAC', 'DT_VAC_MAE',
       'M_AMAMENTA', 'DT_DOSEUNI', 'DT_1_DOSE', 'DT_2_DOSE', 'DT_ANTIVIR',
       'DT_RAIOX', 'DT_COLETA', 'DT_PCR', 'POS_PCRFLU', 'TP_FLU_PCR',
       'PCR_FLUASU', 'FLUASU_OUT', 'PCR_FLUBLI', 'FLUBLI_OUT',
       'POS_PCROUT', 'PCR_VSR', 'PCR_PARA1', 'PCR_PARA2', 'PCR_PARA3',
       'PCR_PARA4', 'PCR_ADENO', 'PCR_METAP', 'PCR_BOCA', 'PCR_RINO',
       'PCR_OUTRO', 'DS_PCR_OUT', 'DT_TOMO', 'TP_TES_AN', 'DT_RES_AN',
       'POS_AN_FLU', 'TP_FLU_AN', 'POS_AN_OUT', 'AN_VSR',
       'AN_PARA1', 'AN_PARA2', 'AN_PARA3', 'AN_ADENO', 'AN_OUTRO',
       'DS_AN_OUT', 'OUT_ANIM', 'LO_PS_VGM', 'DT_VGM', 'DT_RT_VGM',
       'PAIS_VGM', 'CO_PS_VGM', 'CS_ETINIA', 'OUT_SOR', 'SOR_OUT',
       'OUT_ANTIV', 'TOMO_OUT', 'PAC_COCBO', 'PAC_DSCBO', 'DT_CO_SOR',
       'DT_RES', 'TP_AM_SOR', 'TP_SOR', 'DT_SIN_PRI', 'CS_ESCOL_N', 'CO_PAIS', 'ID_PAIS'], axis=1)

In [ ]:
# df = df.drop(['ID_PAIS'], axis=1)

In [ ]:
df[df.columns[16:]].head()

In [ ]:
df.shape

In [ ]:
# Filtrando para utilizar apenas dados de pacientes notificados e internados em SC
df = df.loc[(df['SG_UF_NOT'] == 'SC') & (df['SG_UF_NOT'] == df['SG_UF_INTE']) & (df['CLASSI_FIN'] == 5)]

In [ ]:
# verificando se há dados duplicados
df.duplicated().any()

In [ ]:
# Verificando se, ao definir FATOR_RISC = 'N', os demais campos relacionados a fatores de risco estarão como Ignorado (verifiquei que sim)
df[['FATOR_RISC', 'CARDIOPATI', 'PUERPERA', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI', 'MORB_DESC']]

In [ ]:
# Verificando quantos valores distintos existem para MORB_DESC (verifiquei que são muitos e não daria pra converter para colunas, pois aumentaria muito o número de features)
outrasMorb = pd.Series(df['MORB_DESC'].unique())
print("Qtd: ", outrasMorb.shape[0])
outrasMorb.head(10).to_markdown(tablefmt='html')

In [ ]:
# Preencher como 2-Não nos fatores de risco em branco quando FATOR_RISC = 2-Não
def preencherComBaseEmOutraColuna(linha, colunaBase, valorBase, colunasPreencher, valorPreencher):
    if (linha[colunaBase] == valorBase):
        for colunaPreencher in colunasPreencher:
            if (math.isnan(linha[colunaPreencher])):
                linha[colunaPreencher] = valorPreencher
    return linha

df = df.apply(lambda linha: preencherComBaseEmOutraColuna(linha, 'FATOR_RISC', 'N', ['CARDIOPATI', 'PUERPERA', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI'], 2), axis=1)

In [ ]:
# Verificando valores dos fatores de risco faltantes
friscos = df[['FATOR_RISC', 'CARDIOPATI', 'PUERPERA', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI']]
friscos[friscos.isna().any(axis=1)]

In [ ]:
# como são muitos registros (proporcionalmente ao total de registros disponíveis 3 mil de 17 mil) que estão com NaN, irei considerar que não possui aquele fator de risco que está como Ignorado
df = df.apply(lambda linha: preencherComBaseEmOutraColuna(linha, 'FATOR_RISC', 'S', ['CARDIOPATI', 'PUERPERA', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI'], 2), axis=1)
df[['CARDIOPATI', 'PUERPERA', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI']]

In [ ]:
# confirmando que foi corrigido
friscosCorrigidos = df[['FATOR_RISC', 'CARDIOPATI', 'PUERPERA', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI']]
friscosCorrigidos[friscosCorrigidos.isna().any(axis=1)]

In [ ]:
# verificando percentuais de valores faltantes dos sinais e sintomas
calcularEListarFaltantes(df[['FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'DOR_ABD', 'FADIGA', 'PERD_OLFT', 'PERD_PALA']], 0)

In [ ]:
# Verificando valores dos sinais e sintomas faltantes
sintomas = df[['FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'DOR_ABD', 'FADIGA', 'PERD_OLFT', 'PERD_PALA']]
sintomas[sintomas.isna().any(axis=1)]

In [ ]:
# como são muitos registros com algum sinal/sintoma faltante, considerei que não possui o sintoma
df.fillna(value={'FEBRE': 2, 'TOSSE': 2, 'GARGANTA': 2, 'DISPNEIA': 2, 'DESC_RESP': 2, 'SATURACAO': 2, 'DIARREIA': 2, 'VOMITO': 2, 'DOR_ABD': 2, 'FADIGA': 2, 'PERD_OLFT': 2, 'PERD_PALA': 2}, inplace=True)

In [ ]:
# confirmando o preenchimento
df[['FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'DOR_ABD', 'FADIGA', 'PERD_OLFT', 'PERD_PALA']]

In [ ]:
df.head()

In [ ]:
# Removi as colunas que não seriam utilizadas ou que possuíam muitos valores faltantes que não valeria a pena excluir os registros do dataset
df = df.drop(['SEM_PRI', 'CO_REGIONA', 'ID_UNIDADE', 'CO_UNI_NOT', 'CO_RG_RESI', 'SURTO_SG', 'NOSOCOMIAL', 'OUTRO_DES', 'MORB_DESC', 'TP_ANTIVIR', 'DT_INTERNA', 'CO_RG_INTE',
             'DT_ENTUTI', 'DT_SAIDUTI', 'RAIOX_OUT', 'AMOSTRA', 'TP_AMOSTRA', 'OUT_AMOST', 'CLASSI_OUT', 'CRITERIO', 'DT_EVOLUCA', 'DT_ENCERRA', 'DT_DIGITA', 'CS_GESTANT', 'DT_NOTIFIC', 'DT_NASC'], axis=1)

In [ ]:
# usei este comando para incrementalmente ir removendo as colunas
# df = df.drop(['DT_NASC'], axis=1)

In [ ]:
# Utilizei este comando para verificar coluna por coluna os valores nulos e decidir se excluiria a coluna ou preencheria (e com qual valor)
df.loc[df['SUPORT_VEN'].isnull()]

In [ ]:
df['SUPORT_VEN'].mode()

In [ ]:
# Considerei a moda 1-Zona Urbana
df.fillna({'CS_ZONA':1}, inplace=True)

# Utilizei a moda 2-Não
df.fillna({'AVE_SUINO': 2}, inplace=True)

# Utilizei a moda 2-Não
df.fillna({'OUTRO_SIN': 2}, inplace=True)

# Utilizei a moda 2-Não
df.fillna({'VACINA': 2}, inplace=True)

# Utilizei a moda 2-Não
df.fillna({'ANTIVIRAL': 2}, inplace=True)

# Utilizei a moda 1-Sim
df.fillna({'HOSPITAL': 1}, inplace=True)

# Utilizei a moda 2-Não
df.fillna({'UTI': 2}, inplace=True)

# Utilizei a moda 2-Sim, não invasivo
df.fillna({'SUPORT_VEN': 2}, inplace=True)

# Utilizei a moda 6-Não realizado
df.fillna({'RAIOX_RES': 6}, inplace=True)

# Defini 2-Não, pois geralmente não possui outros dados relacionados também
df.fillna({'AMOSTRA': 2}, inplace=True)

# Utilizei a moda 1-Cura
df.fillna({'EVOLUCAO': 1}, inplace=True)

# Utilizei a moda 2-Não
df.fillna({'HISTO_VGM': 2}, inplace=True)

# Setei para 2-Não pois vazio significa não marcado
df.fillna({'PCR_SARS2': 2}, inplace=True)

# Defini como 6-Não realizado
df.fillna({'TOMO_RES': 6}, inplace=True)

# Setei para 2-Não pois vazio significa não marcado
df.fillna({'AN_SARS2': 2}, inplace=True)

# Usei a moda 1-Branca
df.fillna({'CS_RACA': 1}, inplace=True)

In [ ]:
# Usei este comando para ver aos poucos as colunas e tratar os faltantes com os comandos acima
df[df.columns[57:]].head()

In [ ]:
df.shape

In [ ]:
semIgg = df.drop(['RES_IGG', 'RES_IGM', 'RES_IGA'], axis=1)
semIgg[semIgg.isna().any(axis=1)]

In [ ]:
calcularEListarFaltantes(df, 0)

In [ ]:
# Usei este comando para visualizar alguns registros com determinada coluna vazia
df.loc[df['PCR_RESUL'].isna()]

In [ ]:
# Preenchi com o município de notificação os valores que estavam em branco nos demais (internação e residência)
regionais = df['ID_REGIONA']
df['ID_RG_INTE'] = regionais.where(df['ID_RG_INTE'].isna(), other=df['ID_RG_INTE'])
df['ID_RG_RESI'] = regionais.where(df['ID_RG_RESI'].isna(), other=df['ID_RG_RESI'])

codMunicipios = df['CO_MUN_NOT']
df['CO_MU_INTE'] = codMunicipios.where(df['CO_MU_INTE'].isna(), other=df['CO_MU_INTE'])
df['CO_MUN_RES'] = codMunicipios.where(df['CO_MUN_RES'].isna(), other=df['CO_MUN_RES'])

idsMunicipios = df['ID_MUNICIP']
df['ID_MN_INTE'] = idsMunicipios.where(df['ID_MN_INTE'].isna(), other=df['ID_MN_INTE'])
df['ID_MN_RESI'] = idsMunicipios.where(df['ID_MN_RESI'].isna(), other=df['ID_MN_RESI'])

In [ ]:
df.describe()

In [ ]:
# listando as colunas
pd.Series(df.columns.values).to_markdown(tablefmt="html")

In [ ]:
# Binarização com 0 e 1 ao invés de 1 e 2
df = df.replace(2, {'FEBRE': 0,
               'TOSSE': 0,
               'GARGANTA': 0,
               'DISPNEIA': 0,
               'DESC_RESP': 0,
               'SATURACAO': 0,
               'DIARREIA': 0,
               'VOMITO': 0,
               'DOR_ABD': 0,
               'FADIGA': 0,
               'PERD_OLFT': 0,
               'PERD_PALA': 0,
               'OUTRO_SIN': 0,
               'PUERPERA': 0,
               'CARDIOPATI': 0,
               'HEMATOLOGI': 0,
               'SIND_DOWN': 0,
               'HEPATICA': 0,
               'ASMA': 0,
               'DIABETES': 0,
               'NEUROLOGIC': 0,
               'PNEUMOPATI': 0,
               'IMUNODEPRE': 0,
               'RENAL': 0,
               'OBESIDADE': 0,
               'OUT_MORBI': 0,
               'HISTO_VGM': 0,
               'VACINA': 0,
               'ANTIVIRAL': 0,
               'HOSPITAL': 0,
               'UTI': 0,
               'AVE_SUINO': 0,
               'AN_SARS2': 0,
               'PCR_SARS2': 0})

In [ ]:
df = df.replace('S', {'FATOR_RISC': 1 })
df = df.replace('N', {'FATOR_RISC': 0 })

In [ ]:
df[['SUPORT_VEN']].hist()

In [ ]:
# Tentei fazer isso, mas acho que pode perder uma informação importante.
# Isso porque ao binarizar o campo, a correlação entre ele e o campo UTI diminuiu bastante
#df = df.replace(2, {'SUPORT_VEN': 1 })
#df = df.replace(3, {'SUPORT_VEN': 0 })

In [ ]:
# Aqui buscou-se verificar a quantidade por classificação final do caso
#    1-SRAG por influenza
#    2-SRAG por outro vírus respiratório
#    3-SRAG por outro agente etiológico, qual:
#    4-SRAG não especificado
#    5-SRAG por COVID-19
df[['CLASSI_FIN']].hist()

In [ ]:
corr = df[['FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'OUTRO_SIN', 'PUERPERA', 'CARDIOPATI', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI', 'DOR_ABD', 'UTI', 'SUPORT_VEN']].corr()
corr

In [ ]:
# verificando variáveis mais correlacionadas conforme exemplo do professor Francisco Aparecido Rodrigues, francisco@icmc.usp.br.
# disponível em https://github.com/franciscoicmc/ciencia-de-dados/blob/master/Aula1-Preparacao-transformacao.ipynb

p = 0.75 # correlação mínima
var = []
for i in corr.columns:
    for j in corr.columns:
        if(i != j):
            if np.abs(corr[i][j]) > p: # se maior do que |p|
                var.append([i,j])
print('Variáveis mais correlacionadas:\n', var)

In [ ]:
# distribuição de frequências
def calcularDistribuicaoDeFrequencias(colunas):
    distFreq = pd.DataFrame()
    for c in colunas:
        distFreq[c] = df[c].value_counts(normalize=True)
    return distFreq

display(calcularDistribuicaoDeFrequencias(friscos))
display(calcularDistribuicaoDeFrequencias(sintomas))
display(calcularDistribuicaoDeFrequencias(['CLASSI_FIN']))
display(calcularDistribuicaoDeFrequencias(['SUPORT_VEN']))

In [ ]:
df.head()

In [ ]:
#  RES_AN
#  1-positivo
#  2-Negativo
#  3-Inconclusivo
#  4-Não realizado
#  5-Aguardando resultado
#  9-Ignorado

# AN_SARS2
# 0-Não marcado
# 1-marcado pelo usuário

# PCR_RESUL
# 1-Detectável
# 2-Não Detectável
# 3-Inconclusivo
# 4-Não Realizado
# 5-Aguardando Resultado
# 9-Ignorado

# PCR_SARS2
# 0-Não marcado
# 1-marcado pelo usuário

# CLASSI_FIN
# 1-SRAG por influenza
# 2-SRAG por outro vírus respiratório
# 3-SRAG por outro agente etiológico, qual:
# 4-SRAG não especificado
# 5-SRAG por COVID-19

# Usei esse comando para descobrir como estavam preenchidos os resultados dos exames em relação à classificação final
df[['TOMO_RES', 'RAIOX_RES','RES_AN', 'AN_SARS2', 'PCR_RESUL', 'PCR_SARS2', 'RES_IGG', 'RES_IGM', 'RES_IGA', 'CLASSI_FIN']]

In [ ]:
# Removendo bebês para termos apenas pessoas com idade com valor inteiro em anos
df = df.loc[df['TP_IDADE'] == 3]

In [ ]:
df = df.drop(['RES_AN', 'AN_SARS2', 'PCR_RESUL', 'PCR_SARS2', 'RES_IGG', 'RES_IGM', 'RES_IGA', 'CLASSI_FIN'], axis=1)

In [ ]:
df = df.drop(['TP_IDADE'], axis=1)

In [ ]:
df.head()